In [50]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import metrics
from sklearn.metrics import confusion_matrix, accuracy_score

In [39]:
dt = pd.read_csv('dataset/online_shoppers_intention.csv')

In [40]:
X = dt.iloc[:, :-1]
y = dt.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train.reset_index(inplace = True)
X_test.reset_index(inplace = True)

In [41]:
X_train = X_train.drop(['Administrative', 'Informational', 'ProductRelated'], axis = 1)
X_test = X_test.drop(['Administrative', 'Informational', 'ProductRelated'], axis = 1)

In [42]:
X_train.drop(['OperatingSystems', 'Region', 'TrafficType'], axis = 1, inplace = True)
X_test.drop(['OperatingSystems', 'Region', 'TrafficType'], axis = 1, inplace = True)

In [43]:
labelEncoder = LabelEncoder()
X_train.loc[:, 'Month'] = labelEncoder.fit_transform(X_train.loc[:, 'Month'])
X_test.loc[:, 'Month'] = labelEncoder.transform(X_test.loc[:, 'Month'])
X_train.loc[:, 'VisitorType'] = labelEncoder.fit_transform(X_train.loc[:, 'VisitorType'])
X_test.loc[:, 'VisitorType'] = labelEncoder.transform(X_test.loc[:, 'VisitorType'])
X_train.loc[:, 'Weekend'] = labelEncoder.fit_transform(X_train.loc[:, 'Weekend'])
X_test.loc[:, 'Weekend'] = labelEncoder.transform(X_test.loc[:, 'Weekend'])

C:\Users\kulsh\AppData\Local\Temp\ipykernel_24628\2499845469.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_train.loc[:, 'Month'] = labelEncoder.fit_transform(X_train.loc[:, 'Month'])
C:\Users\kulsh\AppData\Local\Temp\ipykernel_24628\2499845469.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_test.loc[:, 'Month'] = labelEncoder.transform(X_test.loc[:, 'Month'])
C:\Users\kulsh\AppData\Local\Temp\ipykernel_24628\2499845469.py:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inpla

In [44]:
onehotencoder = OneHotEncoder(categories = 'auto', drop = 'first')
train_cat_features = onehotencoder.fit_transform(X_train.loc[:, ['Month', 'Browser', 'VisitorType']]).toarray()
train_cat_features = pd.DataFrame(train_cat_features)
test_cat_features = onehotencoder.transform(X_test.loc[:, ['Month', 'Browser', 'VisitorType']]).toarray()
test_cat_features = pd.DataFrame(test_cat_features)

X_train.drop(['Month', 'Browser', 'VisitorType'], axis = 1, inplace = True)
X_train = X_train.join(train_cat_features)
X_test.drop(['Month', 'Browser', 'VisitorType'], axis = 1, inplace = True)
X_test = X_test.join(test_cat_features)

In [48]:
classifier = Sequential()
classifier.add(Dense(units = 128, activation = 'relu', input_dim = 32))
classifier.add(Dropout(rate = 0.6))
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(rate = 0.6))
classifier.add(Dense(units = 256, activation = 'relu'))
classifier.add(Dropout(rate = 0.6))
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(rate = 0.4))
classifier.add(Dense(units = 1, activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               4224      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               33024     
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               3

In [49]:
history = classifier.fit(X_train, y_train, epochs = 50, shuffle = False, validation_split = 0.1)

Epoch 1/50
278/278 [==============================] - 3s 5ms/step - loss: 86.3254 - accuracy: 0.7230 - val_loss: 1.3091 - val_accuracy: 0.8470
Epoch 2/50
278/278 [==============================] - 1s 4ms/step - loss: 15.5683 - accuracy: 0.7196 - val_loss: 0.6225 - val_accuracy: 0.8480
Epoch 3/50
278/278 [==============================] - 1s 4ms/step - loss: 5.4878 - accuracy: 0.7319 - val_loss: 0.6035 - val_accuracy: 0.8470
Epoch 4/50
278/278 [==============================] - 1s 5ms/step - loss: 2.6600 - accuracy: 0.7514 - val_loss: 0.5459 - val_accuracy: 0.8480
Epoch 5/50
278/278 [==============================] - 1s 5ms/step - loss: 1.6148 - accuracy: 0.7812 - val_loss: 0.4935 - val_accuracy: 0.8480
Epoch 6/50
278/278 [==============================] - 1s 5ms/step - loss: 1.0792 - accuracy: 0.8009 - val_loss: 0.4549 - val_accuracy: 0.8480
Epoch 7/50
278/278 [==============================] - 1s 4ms/step - loss: 0.8263 - accuracy: 0.8146 - val_loss: 0.4339 - val_accuracy: 0.8480
Epoc

In [51]:
y_pred = classifier.predict(X_test)
y_pred = y_pred > 0.5

print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred))
print("Accuracy: {:.2f}%".format(accuracy_score(y_test, y_pred)*100))

78/78 [==============================] - 0s 2ms/step
Confusion matrix:
[[2055    0]
 [ 411    0]]
Accuracy: 83.33%
